# CP321 Project (  Marks)

Student #1:
Name:        Malika Sharma
Student ID:  210 - 37

Student #2:
Name:        Ruveyda Nur Kizmaz
Student ID:  210960430

## Introduction & Motivation

Introduction: xxx

Our Motivation:  xxxxxxx

## Propose Approach

Our Question: What are the trends in student achievements for reading, writing, and mathematics in grades 3, 6, and 9 EQAO assessments across Ontario from 2019 to 2023?

Related Dataset:
https://data.ontario.ca/dataset school-information-and-student-demographics 

Dataset Description:

Data Processing:     ??


Related Graphs we will cover in this file:
- 
-
